In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
import PIL
import read_datasets as rd
import constants as CONST
from importlib import reload
reload(rd)
reload(CONST)

import base64

import pickle
import spacy
import collections
import itertools, collections

import clip
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.datasets import CIFAR100

# Dataloaders

# Prepare model

In [ ]:
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu" # If using GPU then use mixed precision training.
model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training
if device == "cpu":
    model.float()
else :
    clip.model.convert_weights(model) 
    # Actually this line is unnecessary since clip by default already on float16

In [ ]:
loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.Adam(
    model.parameters(), 
    lr=5e-5,
    betas=(0.9,0.98),
    eps=1e-6,
    weight_decay=0.2,
) #Params used from paper, the lr is smaller, more safe for fine tuning to new dataset

In [ ]:
for epoch in range(EPOCH):
    for batch in train_dataloader :
        optimizer.zero_grad()
        
        #list_images is list of image in numpy array(np.uint8), or list of PIL images
        list_image, list_txt = batch 

        images= torch.stack([preprocess(Image.fromarray(img)) for img in list_image],dim=0).to(device) 
        # omit the Image.fromarray if the images already in PIL format, 
        # change this line to images=list_image if using preprocess inside the dataset class
        texts = clip.tokenize(list_txt).to(device)

        logits_per_image, logits_per_text = model(images, texts)

        ground_truth = torch.arange(BATCH_SIZE, dtype=torch.long, device=device)

        total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
        total_loss.backward()
        if device == "cpu":
            optimizer.step()
        else : 
            convert_models_to_fp32(model)
            optimizer.step()
            clip.model.convert_weights(model)